# Get timings

In [114]:
import os
import pandas as pd

list_dataset = ["Lastfm", "Amazon-lb", "QK-video", "Jester", "ML-10M", "ML-20M"]

path_estimate = "../pareto/estimate"
path_full = "../pareto/result_combined"

time_file_estimate = [f for f in os.listdir(path_estimate) if "time" in f]
time_file_full = [f for f in os.listdir(path_full) if "time" in f]

## Full PF and oracle

In [ ]:

result = {}
for f in time_file_full:
    name = f.replace("time_pareto_new_","").split("_")[0]
    if name not in result:
        result[name] = {}
    thetime = pd.read_pickle(f"{path_full}/{f}")
    if "oracle2fair" in f:
        result[name]["oracle2fair"] = thetime
    else:
        result[name]["oracle"] = thetime

df_full = pd.DataFrame(result)
df_full

## Estimation

In [ ]:
est_result = {}
for f in time_file_estimate:
    splitted = f.replace("time_pareto_new_","").split("_")
    name, num_pt_estimate = splitted[0], int(splitted[3].replace("with",""))
    if name not in est_result:
        est_result[name] = {}
    if "oracle2fair" in f:
        est_result[name][f"with{num_pt_estimate+2}"] = pd.read_pickle(f"{path_estimate}/{f}")

       
df_est = pd.DataFrame(est_result)

In [ ]:
df_full.loc["PF"] = df_full.sum()

In [ ]:
final_df = df_full.append(df_est.loc[["with3", "with6", "with12"]])

In [119]:
final_df.loc["Est.~PF (12)"] = final_df.loc["oracle"] + final_df.loc["with12"]
final_df.loc["Est.~PF (6)"] = final_df.loc["oracle"] + final_df.loc["with6"]
final_df.loc["Est.~PF (3)"] = final_df.loc["oracle"] + final_df.loc["with3"]
final_df = final_df.loc[["PF", "Est.~PF (12)", "Est.~PF (6)",  "Est.~PF (3)"], list_dataset]

#TIME IN MINUTES
final_df = final_df / 60

In [ ]:
print(final_df.applymap(lambda x: '{0:.2f}'.format(x)).to_latex())

## MME

### MME - base

load from timing files

In [ ]:
MME_base_files = list(filter(lambda x: "time" in x, os.listdir("../eval/MME")))

In [123]:
MME_base_result = {}
for f in MME_base_files:
    _, _, data, model = f.replace(".pickle","").split("_")
    
    if data not in MME_base_result:
        MME_base_result[data] = {}
    
    
    MME_base_result[data][model] = pd.read_pickle(f"../eval/MME/{f}")


In [ ]:
df_MME = pd.DataFrame(MME_base_result)

### MME - rerank

Have to parse from log

In [ ]:
path_MME_rerank = "../reranking/MME/log"
MME_rerank_files = list(filter(lambda x: "log" in x, os.listdir(path_MME_rerank)))
MME_rerank_files

In [126]:
MME_rerank_result = {}
for f in MME_rerank_files:
    _, _, data, model, rerank = f\
                                    .replace(".txt","")\
                                    .replace("-subset-0.05","")\
                                    .split("_")
    with open(f"{path_MME_rerank}/{f}","r") as thefile:
        line_containing_time = thefile.readlines()[3]
        clean_time = line_containing_time.split(" ")[4].strip("\n")
        clean_time = float(clean_time)

    if data not in MME_rerank_result:
        MME_rerank_result[data] = {}
    
    MME_rerank_result[data][f"{model}_{rerank}"] = clean_time

In [ ]:
df_MME_rerank = pd.DataFrame(MME_rerank_result)

## Combine MME base and rerank

In [ ]:
df_comb_MME = df_MME.append(df_MME_rerank)

In [ ]:
final_df.loc["Total - MME"] = df_comb_MME.sum() / 60
final_df.loc["Avg - MME"] = df_comb_MME.mean() / 60
final_df

## Other joint measures

In [130]:
def get_joint_results(path, rerank=False):
    joint_base_files = list(filter(lambda x: "time" in x, os.listdir(path)))
    joint_base_result = {}

    for f in joint_base_files:
        if not rerank:
            _, measure, data, model = f.replace(".pickle","").split("_")
        elif rerank:
            _, measure, data, model, rerank, _, _= f\
                                                .replace(".pickle","")\
                                                .replace("-subset-0.05","")\
                                                .split("_")
            model = f"{model}_{rerank}"

        if data not in joint_base_result:
            joint_base_result[data] = {}

        if measure not in joint_base_result[data]:
            joint_base_result[data][measure] = {}
        
        joint_base_result[data][measure][model] = pd.read_pickle(f"{path}/{f}")

    df_joint = pd.DataFrame(joint_base_result)

    return df_joint


In [ ]:
base_joint = get_joint_results("../eval/joint")

In [132]:
path_joint_rerank = "../reranking/joint/"
rerank_joint = get_joint_results(path_joint_rerank, rerank=True)


In [ ]:
rerank_joint = rerank_joint.rename(index={"IAArerank":"IAA"})

In [134]:
def merge(dict1, dict2):
    res = {**dict1, **dict2}
    return res

In [ ]:
df_joint = base_joint.append(rerank_joint)
df_joint = df_joint.groupby(df_joint.index).agg(lambda x: x)

In [ ]:
df_joint = df_joint.applymap(lambda x: merge(x[0], x[1]))

In [ ]:
df_joint.applymap(len)

In [138]:
df_joint_avg = df_joint.applymap(lambda x: sum(x.values())/len(x))
df_joint_sum = df_joint.applymap(lambda x: sum(x.values()))

In [ ]:
df_joint_avg = df_joint_avg.rename(index={"AIF":"AI-F","IIF":"II-F"})
df_joint_avg = df_joint_avg / 60

In [140]:
df_joint_avg.index = "Avg - " + df_joint_avg.index

In [141]:
df_joint_sum = df_joint_sum.rename(index={"AIF":"AI-F","IIF":"II-F"})
df_joint_sum = df_joint_sum / 60
df_joint_sum.index = "Total - " + df_joint_sum.index

In [142]:
names = ["IBO", "MME", "IAA", "II-F", "AI-F"]
avg_names = [f"Avg - {name}" for name in names]
sum_names = [f"Total - {name}" for name in names]

In [ ]:
final_df = final_df.append(df_joint_avg).append(df_joint_sum)
final_df = final_df.loc[["PF", "Est.~PF (12)", "Est.~PF (6)"] + avg_names + sum_names]

## Latex table from `final_df`

In [ ]:
final_df_rounded = final_df.round(2)
final_df_rounded

In [189]:
final_df_rounded[final_df_rounded == 0]  = '<0.3s'

In [ ]:
print(final_df_rounded.round(2).to_latex(escape=False))

# Compare estimated PF to PF

In [103]:
import pandas as pd
import numpy as np

efficiency_table = {}
base_path = "../pareto"
full_path = "/result_combined"
est_path = "/estimate"

list_dataset = [
                "Lastfm", 
                "Amazon-lb", "QK-video", 
                "Jester", 
                "ML-10M", 
                "ML-20M"
                ]

should_decrease_measures = [
                            'P@10', 'MAP@10', 'R@10', 'NDCG@10', 'Gini_our@10',]
should_increase_measures = [
                            'Jain_our@10', 
                            'Ent_our@10', 
                            ]

def check_monotonic(x:pd.Series):
    if x.name in should_decrease_measures:
        return x.is_monotonic_decreasing
    elif x.name in should_increase_measures:
        return x.is_monotonic_increasing

for dataset in list_dataset:
    efficiency_table[dataset] = {}
    real = pd.read_pickle(f"{base_path}{full_path}/pareto_new_{dataset}_oraclefair_at10.pickle")
    #here we dont drop duplicates
    # we take all the points in the PF including those that may be duplicates (because the duplicates may be unique per measure pair)
    efficiency_table[dataset]["\# points in full PF"] = int(real.shape[0]) 
    efficiency_table[dataset]["\% points in estimated PF"] = []

    for numpt in range(1,11):
        # print(numpt)
        est = pd.read_pickle(f"{base_path}{est_path}/estimate_new_{dataset}_oraclefair_at10_with{numpt}.pickle")
        
    #check if all points in est are in real
    #thanks to https://stackoverflow.com/questions/49530918/check-if-pandas-dataframe-is-subset-of-other-dataframe
        try:
            assert len(est.merge(real, how="inner").drop_duplicates()) == len(est.drop_duplicates())
            assert len(est) == (numpt+2)
        except:
            print(dataset, numpt, len(est))

        tocheck = real.loc[:,(real.columns.isin(should_decrease_measures)|real.columns.isin(should_increase_measures))]

        try:
            assert all(tocheck.apply(lambda x: check_monotonic(x)))
        except:
            print(f"One or more columns are not monotonic in {dataset}")
            display(tocheck.apply(lambda x: check_monotonic(x)))
    
        pctg_est_point_out_of_full = est.shape[0] / real.shape[0] * 100

        efficiency_table[dataset]["\% points in estimated PF"].append(pctg_est_point_out_of_full)


In [ ]:
df_efficiency = pd.DataFrame(efficiency_table).T

In [ ]:
print(pd.DataFrame(df_efficiency["\\% points in estimated PF"].apply(min).apply(lambda x: '{0:.2f}'.format(x))).T.to_latex())

In [ ]:
print(pd.DataFrame(df_efficiency["\\% points in estimated PF"].apply(max).round(2).apply(lambda x: '{0:.2f}'.format(x))).T.to_latex())

In [ ]:
print(pd.DataFrame(df_efficiency["\\% points in estimated PF"].apply(lambda x: x[3]).round(2).apply(lambda x: '{0:.2f}'.format(x))).T.to_latex())

In [36]:
def get_range(list_of_val, rounding=3):
    min_val = min(list_of_val)
    max_val = max(list_of_val)
    return f"{'{0:.3f}'.format(round(min_val,rounding))}--{round(max_val,rounding)}"

In [37]:
df_efficiency["\\% points in estimated PF"] = df_efficiency["\\% points in estimated PF"].apply(get_range)

In [38]:
result_est = {}
path_integral_point_full = pd.read_pickle("efficiency/path_integral_point_full.pickle")

In [39]:
def filter_path_integral_point(path_integral_point):
    path_integral_point = path_integral_point.loc[path_integral_point.index.str.contains("our")]
    path_integral_point = path_integral_point.loc[path_integral_point.index.str.contains("Jain|Ent|Gini")]
    path_integral_point = path_integral_point.drop(columns=["HR@10","MRR@10"])
    return path_integral_point

def compute_avg_delta_position(path_integral_full_d, path_integral_est_d):
    diff = path_integral_full_d - path_integral_est_d
    final_res = diff\
                    .applymap(lambda x: np.power(x, 2))\
                    .applymap(sum)\
                    .applymap(np.sqrt)\
                    .stack()\
                    .dropna()\
                    .mean()
    
    return final_res

for data in list_dataset:
    path_integral_full_data = filter_path_integral_point(path_integral_point_full[data])
    result_est[data] = []
    for numpt in range(1,11):
        path_integral_point_est = pd.read_pickle(f"efficiency/path_integral_point_with{numpt}.pickle")
        path_integral_est_data = filter_path_integral_point(path_integral_point_est[data])

        euclid_distance = compute_avg_delta_position(path_integral_full_data, path_integral_est_data)

        result_est[data].append(euclid_distance)

In [86]:
dist_6 = {}
dist_12 = {}
dist_3 = {}
for k, v in result_est.items():
    dist_3[k] = v[0]
     #index 3 and 9, because numpoint is 4 and 10 , numpoint in pareto = 6 and 12
    dist_6[k] = v[3]
    dist_12[k] = v[9]

In [87]:
row_dist_6 = pd.Series(dist_6)
row_dist_12 = pd.Series(dist_12)
row_dist_3 = pd.Series(dist_3)

In [88]:
newdf = pd.DataFrame()
newdf["dist3"] = row_dist_3
newdf["dist6"] = row_dist_6
newdf["dist12"] = row_dist_12

In [ ]:
print(newdf.T.applymap(lambda x: '{0:.2f}'.format(x)).to_latex())

# Corr table

In [81]:
model_distance_dict_full = pd.read_pickle(f"efficiency/model_distance_dict_full.pickle")

In [82]:
def distance_based_rank_for_corr(model_distance_dict, data):
    rank_based_on_distance = model_distance_dict[data].unstack().reset_index()
    rank_based_on_distance.columns = ["rel","fair","models"]
    rank_based_on_distance = rank_based_on_distance.loc[rank_based_on_distance.rel.str.contains("^P|^R|NDCG|MAP")]
    rank_based_on_distance = rank_based_on_distance.loc[rank_based_on_distance.fair.str.contains("Jain|Gini|Ent")]
    rank_based_on_distance = rank_based_on_distance.loc[rank_based_on_distance.fair.str.contains("our")]
    rank_based_on_distance = rank_based_on_distance.loc[rank_based_on_distance.models.apply(lambda x: x[1]).dropna().index]
    rank_based_on_distance["col_name"] = rank_based_on_distance.rel + "-" + rank_based_on_distance.fair
    rank_based_on_distance = rank_based_on_distance[["col_name","models"]].T
    rank_based_on_distance.columns = rank_based_on_distance.loc["col_name"]
    rank_based_on_distance = rank_based_on_distance.iloc[1].T
    
    dict_rank_based_on_distance = {}

    for row, item in pd.DataFrame(rank_based_on_distance).iterrows():
        the_tup = item[0]
        model_name = the_tup[0]
        scores = the_tup[1]
        dict_rank_based_on_distance[row] = dict((key,val) for key,val in zip(model_name, scores))


    for_corr = pd.DataFrame(dict_rank_based_on_distance).T.applymap(lambda x: -x)

    return for_corr


def compare_full_est(model_distance_dict_full, model_distance_dict_est):

    forplot = pd.DataFrame()

    for data in list_dataset:
        for_corr_full = distance_based_rank_for_corr(model_distance_dict_full, data)
        for_corr_est = distance_based_rank_for_corr(model_distance_dict_est, data)

        for_corr_full.index = for_corr_full.index.str.replace("_our@10", "") + "-full"
        for_corr_est.index = for_corr_est.index.str.replace("_our@10", "") + "-est"

        final_corr = pd.concat([for_corr_full, for_corr_est])
        
        final_corr = final_corr.T\
                        .reset_index(drop=True)\
                        .applymap(float)\
                        .corr(method="kendall")\
                        .round(2)
        
        filtered_final_corr = final_corr.loc[final_corr.index[final_corr.index.str.contains("full")], final_corr.columns[final_corr.columns.str.contains("est")]]

        forplot[data] = pd.Series(np.diag(filtered_final_corr), index=[filtered_final_corr.index.str.replace("-full","")]) 
    

    return forplot

result_dict = {}

for numpt in range(1, 11):
    model_distance_dict_est = pd.read_pickle(f"efficiency/model_distance_dict_with{numpt}.pickle")
    fullvsest = compare_full_est(model_distance_dict_full, model_distance_dict_est)

    result_dict[numpt+2] = fullvsest.min().apply(lambda x: '{0:.2f}'.format(x)) +  "--" + fullvsest.max().apply(lambda x: '{0:.2f}'.format(x))  

In [ ]:
#appendix
print(pd.DataFrame(result_dict).T.to_latex()) 

In [84]:
result_dict = {}

for numpt in range(1, 11):
    model_distance_dict_est = pd.read_pickle(f"efficiency/model_distance_dict_with{numpt}.pickle")
    fullvsest = compare_full_est(model_distance_dict_full, model_distance_dict_est)

    result_dict[numpt+2] = fullvsest.min()

In [ ]:
#main paper
print(pd.DataFrame(result_dict).T.to_latex()) 